In [1]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute, IBMQ, Aer

In [ ]:
APItoken='your APItoken'

In [ ]:
IBMQ.enable_account(APItoken)

### simulator

In [2]:
simulator = Aer.get_backend('qasm_simulator')
#simulator

### real device

In [ ]:
provider= IBMQ.get_provider()
provider.backends()
#get provider first

In [ ]:
backend=provider.get_backend('ibmq_16_melbourne')

## Obtaining the Pt3 for the quantum witness

In [3]:
def Pt3(t):
    q=QuantumRegister(7)
    c=ClassicalRegister(6)
    qc=QuantumCircuit(q,c)
    qc.h(q[1])
    qc.cx(q[1],q[2])
    qc.cx(q[2],q[3])
    qc.cx(q[3],q[4])
    qc.cx(q[4],q[5])
    qc.cx(q[5],q[6])
    #produce the GHZ state
    
    qc.barrier()
    # If we don't add the barrier, IBM Q will automatically combine the UU^{dagger}=I . 
    # And the IBM would not perform the real processing
    qc.cx(q[5],q[6])
    qc.cx(q[4],q[5])
    qc.cx(q[3],q[4])
    qc.cx(q[2],q[3])
    qc.cx(q[1],q[2])
    qc.h(q[1])
    qc.measure(q[1],c[0])
    qc.measure(q[2],c[1])
    qc.measure(q[3],c[2])
    qc.measure(q[4],c[3])
    qc.measure(q[5],c[4])
    qc.measure(q[6],c[5])
    #assign the number of each qubits
    return qc

## By using the ancilla qubit, we can obtain the two-time correlation function by the following code (Direct-measre scenario)

In [4]:
def Pt2t3(t):
    q=QuantumRegister(14)
    c=ClassicalRegister(12)
    qc=QuantumCircuit(q,c)
    qc.h(q[1])
    qc.cx(q[1],q[2])
    qc.cx(q[2],q[3])
    qc.cx(q[3],q[4])
    qc.cx(q[4],q[5])
    qc.cx(q[5],q[6])
    #create GHZ state
    qc.cx(q[1],q[13])
    qc.cx(q[2],q[12])
    qc.cx(q[3],q[11])
    qc.cx(q[4],q[10])
    qc.cx(q[5],q[9])
    qc.cx(q[6],q[8])
    #q[13] q[12] q[11] q[10] q[9] q[8] are ancilla qubits
    #they are used to measure the GHZ state containing q[1] q[2] q[3] q[4] q[5] at t2 respectively
    qc.cx(q[5],q[6])
    qc.cx(q[4],q[5])
    qc.cx(q[3],q[4])
    qc.cx(q[2],q[3])
    qc.cx(q[1],q[2])
    qc.h(q[1])
    qc.measure(q[13],c[0])
    qc.measure(q[12],c[1])
    qc.measure(q[11],c[2])
    qc.measure(q[10],c[3])
    qc.measure(q[9],c[4])
    qc.measure(q[8],c[5])
    qc.measure(q[1],c[6])
    qc.measure(q[2],c[7])
    qc.measure(q[3],c[8])
    qc.measure(q[4],c[9])
    qc.measure(q[5],c[10])
    qc.measure(q[6],c[11])
    return qc

# Running the quantum circuit by IBM and arrange the raw data

In [9]:
A=[]
for t in range(5):
    job=execute(Pt3(0),simulator,shots=8192)
    A.append(job)

In [11]:
for i in range(5):
    print(A[i].result().get_counts())

{'000000': 8192}
{'000000': 8192}
{'000000': 8192}
{'000000': 8192}
{'000000': 8192}


# pick Pt3

In [14]:
f={}
f[0]={'000000': 8192}
f[1]={'000000': 8192}
f[2]={'000000': 8192}
f[3]={'000000': 8192}
f[4]={'000000': 8192}

In [18]:
import copy
def pick_t3(g,k):
    f=copy.deepcopy(g)
    for i in range(len(f)):
    #Transfre the result become probabilty form
        for key in f[i].keys():
            f[i][key] = f[i][key]/8192
            if key == k:
            # n is the data we want for Pt3
                print(f[i][k])

In [19]:
pick_t3(f,'000000')

1.0
1.0
1.0
1.0
1.0


# Get Pt2t3

In [25]:
A=[]
for t in range(5):
    job=execute(Pt2t3(0),simulator,shots=8192)
    A.append(job)

In [26]:
for i in range(5):
    print(A[i].result().get_counts())

{'000001000000': 2066, '000001111111': 2081, '000000111111': 1987, '000000000000': 2058}
{'000001000000': 2072, '000001111111': 2011, '000000111111': 2014, '000000000000': 2095}
{'000001000000': 2019, '000001111111': 2084, '000000111111': 2025, '000000000000': 2064}
{'000001000000': 2058, '000001111111': 2085, '000000111111': 1994, '000000000000': 2055}
{'000001000000': 1991, '000001111111': 2087, '000000111111': 2024, '000000000000': 2090}


In [27]:
f={}
f[0]={'000001000000': 2066, '000001111111': 2081, '000000111111': 1987, '000000000000': 2058}
f[1]={'000001000000': 2072, '000001111111': 2011, '000000111111': 2014, '000000000000': 2095}
f[2]={'000001000000': 2019, '000001111111': 2084, '000000111111': 2025, '000000000000': 2064}
f[3]={'000001000000': 2058, '000001111111': 2085, '000000111111': 1994, '000000000000': 2055}
f[4]={'000001000000': 1991, '000001111111': 2087, '000000111111': 2024, '000000000000': 2090}

In [22]:
import copy
def picks(g,n,k):
    # g is the raw data
    # n represents the number of the entangled qubits you used
    # k is the final_state you want from the raw data
    effective={}
    f=copy.deepcopy(g)
    # we don't want the data to be changed, so we make a copy of that
    for i in range(len(f)):
        for key in f[i].keys():
            f[i][key] = f[i][key]/8192
        data=f[i].keys()
        state=[]
        for j in data:
            state.append(j)
        final_state=[u[0:n] for u in state]
        effective_state=[]
        for j in range(len(state)):
            if final_state[j] == k :
                effective_state.append(state[j])
                    
        effective[i]={key:f[i][key] for key in effective_state}
        
    return effective

In [28]:
picks(f,6,'000000')
# the sum of data is Pt2*Pt2t3

{0: {'000000111111': 0.2425537109375, '000000000000': 0.251220703125},
 1: {'000000111111': 0.245849609375, '000000000000': 0.2557373046875},
 2: {'000000111111': 0.2471923828125, '000000000000': 0.251953125},
 3: {'000000111111': 0.243408203125, '000000000000': 0.2508544921875},
 4: {'000000111111': 0.2470703125, '000000000000': 0.255126953125}}

# We can get the witness by |Pt3 - sum(Pt2*Pt2t3)|